In [4]:
import os
os.chdir("Ruta/a/tu/carpeta")

import pandas as pd
import geopandas as gpd

gdf=gpd.read_file('rawdata/Manzanas_Chimbote_INEI_Oficial.geojson')
gdf.head()

,fid,UBIGEO,CODDPTO,NOMBDPTO,CODPROV,NOMBPROV,CODDIST,NOMBDIST,CODCCPP,NOMCCPP,CODZONA,SUFZONA,CODMZNA,SUFMZNA,CIUDAD,IDMANZANA,ID,geometry
0,1,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,060,U,CHIMBOTE,02180900500060U,1,"MULTIPOLYGON (((-78.50067 -9.12268, -78.50088 ..."
1,2,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,060,T,CHIMBOTE,02180900500060T,2,"MULTIPOLYGON (((-78.50043 -9.12242, -78.50064 ..."
2,3,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,062,None,CHIMBOTE,02180900500062,3,"MULTIPOLYGON (((-78.49951 -9.12266, -78.49966 ..."
3,4,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,060,S,CHIMBOTE,02180900500060S,4,"MULTIPOLYGON (((-78.50022 -9.12221, -78.50040 ..."
4,5,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,060,V,CHIMBOTE,02180900500060V,5,"MULTIPOLYGON (((-78.49914 -9.12227, -78.49932 ..."


Lo que se ve es que en este gdf que me pasa INEI por correo es que el ID de manzana está compuesto por *UBIGEO + CODZONA + SUFZONA + CODMZNA + SUFMZNA*. 

Sin embargo, cuando uno baja los datos de REDATAM, el ID de manzana está compuesto por *UBIGEO + **CODCCPP** + CODZONA + SUFZONA + CODMZNA + SUFMZNA*. Por tanto, lo que hemos hecho en el cuaderno *1-Recalculando el Codigo de Manzana* ha sido calcular ese nuevo código de manzana para añadirlo al geodataframe y luego poder jalar las variables de REDATAM sin problema.

In [8]:
idmz=pd.read_csv("data/CodigoManzanasOficial.csv")
gdf_nuevo=gdf.drop(["IDMANZANA",              #excluimos la columna que ya no usaremos
                   "fid"],                    #usamos la columna ID como referencia, esta está de más
                   axis=1) 
gdf_nuevo = gpd.GeoDataFrame(gdf_nuevo.merge(idmz, on='ID', how='left'))
gdf_nuevo.head()

,UBIGEO,CODDPTO,NOMBDPTO,CODPROV,NOMBPROV,CODDIST,NOMBDIST,CODCCPP,NOMCCPP,CODZONA,SUFZONA,CODMZNA,SUFMZNA,CIUDAD,ID,geometry,Unnamed: 0,Mz
0,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,060,U,CHIMBOTE,1,"MULTIPOLYGON (((-78.50067 -9.12268, -78.50088 ...",1,021809000100500060U
1,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,060,T,CHIMBOTE,2,"MULTIPOLYGON (((-78.50043 -9.12242, -78.50064 ...",2,021809000100500060T
2,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,062,None,CHIMBOTE,3,"MULTIPOLYGON (((-78.49951 -9.12266, -78.49966 ...",3,021809000100500062
3,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,060,S,CHIMBOTE,4,"MULTIPOLYGON (((-78.50022 -9.12221, -78.50040 ...",4,021809000100500060S
4,021809,02,ANCASH,18,SANTA,09,NUEVO CHIMBOTE,0001,BUENOS AIRES,005,00,060,V,CHIMBOTE,5,"MULTIPOLYGON (((-78.49914 -9.12227, -78.49932 ...",5,021809000100500060V


In [14]:
list(gdf_nuevo.columns.values)

['UBIGEO',
 'CODDPTO',
 'NOMBDPTO',
 'CODPROV',
 'NOMBPROV',
 'CODDIST',
 'NOMBDIST',
 'CODCCPP',
 'NOMCCPP',
 'CODZONA',
 'SUFZONA',
 'CODMZNA',
 'SUFMZNA',
 'CIUDAD',
 'ID',
 'geometry',
 'Mz']

In [15]:
gdf_nuevo=gdf_nuevo.drop("Unnamed: 0",axis=1)
gdf_nuevo.to_file("data/1-Mzs_Chimbote_Oficial_INEI.geojson",driver='GeoJSON')